In [1]:
from relforte import *

In [1]:
mol = pyscf.gto.M(
    verbose = 2,
    atom = '''
H 0 0 0
F 0 1.5 0
''',
    basis = 'cc-pvdz', spin=0, charge=0, symmetry=False
)
hf = RelForte(mol, verbose=True, density_fitting=False, decontract=False)
hf.run_dhf(transform=True, debug=False, frozen=(0,0), dump_mo_coeff=False)
hf.run_casci(cas=(6,8), do_fci=False, rdm_level=3, relativistic=True, semi_canonicalize=True)
hf.run_dsrg_mrpt3(s=2.0, relativistic=True, relax=None)
try:
    assert np.isclose(hf.e_dsrg_mrpt3_1, 0.003323575826786)
except:
    print('Warning: DSRG-MRPT3 energy part 1 is not close to reference value.')
try:
    assert np.isclose(hf.e_dsrg_mrpt2, -0.198886593262808)
except:
    print('Warning: DSRG-MRPT2 energy is not close to reference value.')
try:
    assert np.isclose(hf.e_dsrg_mrpt3_2, -0.004129141541982)
except:
    print('Warning: DSRG-MRPT3 energy part 2 is not close to reference value.')
try:
    assert np.isclose(hf.e_dsrg_mrpt3_3, 0.000009041115140)
except:
    print('Warning: DSRG-MRPT3 energy part 3 is not close to reference value.')

              PySCF DHF interface              
Relativistic DHF Energy:         -99.9638776 Eh
PySCF RHF time:                    0.6612122 s
-----------------------------------------------
Will now allocate 66.724 MB disk space for the AO ERI tensor!

Timing report
....integral transformation:       3.2533035 s
Total time taken:                  3.2537415 s

            Relativistic CASCI(6,8)            
Will now allocate 0.013 MB memory for the CASCI Hamiltonian!
Time taken for 1-RDM build:        0.0043848 s
Time taken for 2-RDM build:        0.0167646 s
Time taken for 3-RDM build:        0.1167014 s
E_frzc:                          -88.8825774 Eh
E_cas:                           -14.2940434 Eh
E_nuc:                             3.1750633 Eh
E_casci:                        -100.0015576 Eh
E0 (from RDM):                  -100.0015576 Eh
Ecorr:                            -0.0376800 Eh

Timing summary
... Hamil build:                    0.0125654 s
... Hamil diag:                    

In [19]:
F_1_tilde = np.zeros(hf.fock.shape, dtype='complex128')
V_1_tilde = np.zeros(hf.dhf_eri_full_asym.shape, dtype='complex128')
F_1_tilde[hf.hole, hf.part] = hf.fock[hf.hole, hf.part].conj() + (hf.fock[hf.hole, hf.part].conj() + np.einsum('xu,iuax,xu->ia', hf.denom_act, hf.T2_1[:,hf.ha,:,hf.pa],hf.cumulants['gamma1']))*hf.d1_exp
V_1_tilde[hf.hole, hf.hole, hf.part, hf.part] = hf.dhf_eri_full_asym[hf.hole, hf.hole, hf.part, hf.part].conj() + (hf.dhf_eri_full_asym[hf.hole, hf.hole, hf.part, hf.part].conj())*hf.d2_exp

F_1_tilde = F_1_tilde.T
V_1_tilde = np.einsum('ijab->abij', V_1_tilde)

In [20]:
fixed_args = (hf.cumulants['gamma1'], hf.cumulants['eta1'], hf.cumulants['lambda2'], hf.cumulants['lambda3'], hf)
H_T_C0(None, None, F_1_tilde, V_1_tilde, hf.T1_1, hf.T2_1, *fixed_args)

(-0.07872062351658052+0.01999955146143875j)

In [31]:
F_1_tilde = np.zeros(hf.fock.shape, dtype='complex128')
F_1_tilde[hf.hole,hf.part] = np.copy(hf.fock[hf.hole,hf.part].conj())
F_1_tilde[hf.hole,hf.part] += F_1_tilde[hf.hole,hf.part] * hf.d1_exp
F_1_tilde[hf.hole,hf.part] += np.multiply(hf.d1_exp, np.einsum('xu,iuax,xu->ia',hf.denom_act,hf.T2_1[:,hf.ha,:,hf.pa],hf.cumulants['gamma1']))
F_1_tilde = np.conj(F_1_tilde)

V_1_tilde = np.zeros(hf.dhf_eri_full_asym.shape, dtype='complex128')
V_1_tilde[hf.hole,hf.hole,hf.part,hf.part] = hf.dhf_eri_full_asym[hf.hole,hf.hole,hf.part,hf.part].copy()
V_1_tilde[hf.hole,hf.hole,hf.part,hf.part] += V_1_tilde[hf.hole,hf.hole,hf.part,hf.part] * hf.d2_exp

e_dsrg_mrpt2 = H_T_C0(None, None, F_1_tilde.T, np.einsum('ijab->abij',V_1_tilde), hf.T1_1, hf.T2_1, *fixed_args)
print(e_dsrg_mrpt2)


(-0.1988865932578673-1.613960094032669e-17j)


In [16]:
fixed_args = (hf.cumulants['gamma1'], hf.cumulants['eta1'], hf.cumulants['lambda2'], hf.cumulants['lambda3'], hf)

Htilde1_1b = hf.H0A1_1b + 2*hf.F1.conj()
Htilde1_2b = hf.H0A1_2b + 2*hf.dhf_eri_full_asym.conj()

Htilde1A1_1b = np.zeros((hf.nlrg,hf.nlrg), dtype='complex128')
Htilde1A1_2b = np.zeros((hf.nlrg,hf.nlrg,hf.nlrg,hf.nlrg), dtype='complex128')

H1_T1_C1(Htilde1A1_1b, None, Htilde1_1b, None,          hf.T1_1, None,    *fixed_args)
H1_T2_C1(Htilde1A1_1b, None, Htilde1_1b, None,          None,    hf.T2_1, *fixed_args)
H2_T1_C1(Htilde1A1_1b, None, None, Htilde1_2b, hf.T1_1, None,    *fixed_args)
H2_T2_C1(Htilde1A1_1b, None, None, Htilde1_2b, None,    hf.T2_1, *fixed_args)
H1_T2_C2(None, Htilde1A1_2b, Htilde1_1b, None,          None,    hf.T2_1, *fixed_args)
H2_T1_C2(None, Htilde1A1_2b, None, Htilde1_2b, hf.T1_1, None,    *fixed_args)
H2_T2_C2(None, Htilde1A1_2b, None, Htilde1_2b, None,    hf.T2_1, *fixed_args)
Htilde1A1_2b = antisymmetrize_and_hermitize(Htilde1A1_2b)# - 0.5*H0A1A1_2b
Htilde1A1_1b = (Htilde1A1_1b + Htilde1A1_1b.T.conj())# - 0.5*H0A1A1_1b
T1_2, T2_2 = hf.form_amplitudes(Htilde1A1_1b, Htilde1A1_2b)
E = 0.5*H_T_C0(None, None, Htilde1_1b, Htilde1_2b, T1_2, T2_2, *fixed_args)
print(f'DSRG-MRPT3 energy pt 2: {E:.15f}')

DSRG-MRPT3 energy pt 2: -0.004129141543266+0.000000000000000j


In [ ]:
hf.e_dsrg_mrpt2 = dsrg_HT(hf.F_1_tilde, hf.V_1_tilde, hf.T1_1, hf.T2_1, hf.cumulants['gamma1'], hf.cumulants['eta1'], hf.cumulants['lambda2'], hf.cumulants['lambda3'], hf)
print(hf.e_dsrg_mrpt2)

(-0.19888659325351396+4.084434935566394e-18j)


In [ ]:
H_T_C0(hf.F_1_tilde, hf.V_1_tilde, hf.T1_1, hf.T2_1, hf.cumulants['gamma1'], hf.cumulants['eta1'], hf.cumulants['lambda2'], hf.cumulants['lambda3'], hf)

In [ ]:
t0 = time.time()
fixed_args = (hf.cumulants['gamma1'], hf.cumulants['eta1'], hf.cumulants['lambda2'], hf.cumulants['lambda3'], hf)

H0A1_1b = np.zeros((hf.nlrg,hf.nlrg), dtype='complex128')
H0A1_2b = np.zeros((hf.nlrg,hf.nlrg,hf.nlrg,hf.nlrg), dtype='complex128')
H1_T2_C2(None, H0A1_2b, hf.F0, None, None, hf.T2_1, *fixed_args)
H1_T2_C1(H0A1_1b, None, hf.F0, None, None, hf.T2_1, *fixed_args)
H1_T1_C1(H0A1_1b, None, hf.F0, None, hf.T1_1, None, *fixed_args)
H0A1_2b = antisymmetrize_and_hermitize(H0A1_2b)
H0A1_1b = H0A1_1b + H0A1_1b.T.conj()

H0A1A1_1b = np.zeros((hf.nlrg,hf.nlrg), dtype='complex128')
H0A1A1_2b = np.zeros((hf.nlrg,hf.nlrg,hf.nlrg,hf.nlrg), dtype='complex128')

H1_T1_C1(H0A1A1_1b, None,      H0A1_1b, None,    hf.T1_1, None,    *fixed_args)
H1_T2_C1(H0A1A1_1b, None,      H0A1_1b, None,    None,    hf.T2_1, *fixed_args)
H2_T1_C1(H0A1A1_1b, None,      None,    H0A1_2b, hf.T1_1, None,    *fixed_args)
H2_T2_C1(H0A1A1_1b, None,      None,    H0A1_2b, None,    hf.T2_1, *fixed_args)
H1_T2_C2(None,      H0A1A1_2b, H0A1_1b, None,    None,    hf.T2_1, *fixed_args)
H2_T1_C2(None,      H0A1A1_2b, None,    H0A1_2b, hf.T1_1, None,    *fixed_args)
H2_T2_C2(None,      H0A1A1_2b, None,    H0A1_2b, None,    hf.T2_1, *fixed_args)
H0A1A1_2b = antisymmetrize_and_hermitize(H0A1A1_2b)
H0A1A1_1b = H0A1A1_1b + H0A1A1_1b.T.conj()
H0A1A1_1b = -H0A1A1_1b
H0A1A1_2b = -H0A1A1_2b

t1 = time.time()
print("Time for Hbar: ", t1-t0)

E = (1./6) * H_T_C0(None, None, H0A1A1_1b, H0A1A1_2b, hf.T1_1, hf.T2_1, *fixed_args)
print(f'DSRG-MRPT3 energy pt 1: {E:.15f}')


Time for Hbar:  0.509028434753418
DSRG-MRPT3 energy pt 1: 0.003323575826344+0.000000000000000j


In [ ]:
mol = pyscf.gto.M(
    verbose = 2,
    atom = '''
H 0 0 0
F 0 1.5 0
''',
    basis = 'cc-pvdz', spin=0, charge=0, symmetry=False
)
hf_pt2 = RelForte(mol, verbose=True, density_fitting=False, decontract=False)
hf_pt2.run_dhf(transform=True, debug=False, frozen=(0,0), dump_mo_coeff=False)
hf_pt2.run_casci(cas=(6,8), do_fci=False, rdm_level=3, relativistic=True, semi_canonicalize=True)
hf_pt2.run_dsrg_mrpt2(s=2.0, relativistic=True, relax=None)

              PySCF DHF interface              
Relativistic DHF Energy:         -99.9638776 Eh
PySCF RHF time:                    0.8191667 s
-----------------------------------------------
Will now allocate 66.724 MB disk space for the AO ERI tensor!

Timing report
....integral transformation:       3.8645244 s
Total time taken:                  3.8650498 s

            Relativistic CASCI(6,8)            
Will now allocate 0.013 MB memory for the CASCI Hamiltonian!
Time taken for 1-RDM build:        0.0060654 s
Time taken for 2-RDM build:        0.0176573 s
Time taken for 3-RDM build:        0.1317499 s
E_frzc:                          -88.8825774 Eh
E_cas:                           -14.2940434 Eh
E_nuc:                             3.1750633 Eh
E_casci:                        -100.0015576 Eh
E0 (from RDM):                  -100.0015576 Eh
Ecorr:                            -0.0376800 Eh

Timing summary
... Hamil build:                    0.0127583 s
... Hamil diag:                    

In [ ]:
hf.T2_1 = hf_pt2.T2_1
hf.T1_1 = hf_pt2.T1_1
hf.cumulants = hf_pt2.cumulants

In [ ]:
hf.compute_energy_pt3_1()

DSRG-MRPT3 energy pt 1: 0.002632797241654-0.000000000000000j
Time taken:  0.47235846519470215


In [ ]:
fixed_args = (hf_pt2.cumulants['gamma1'], hf_pt2.cumulants['eta1'], hf_pt2.cumulants['lambda2'], hf_pt2.cumulants['lambda3'], hf_pt2)

H0A1_1b = np.zeros((hf.nlrg,hf.nlrg), dtype='complex128')
H0A1_2b = np.zeros((hf.nlrg,hf.nlrg,hf.nlrg,hf.nlrg), dtype='complex128')
compute_ht_h1_only(fixed_args, H0A1_1b, H0A1_2b, hf.F0, hf_pt2.T1_1, hf_pt2.T2_1)

H0A1A1_1b = np.zeros((hf_pt2.nlrg,hf_pt2.nlrg), dtype='complex128')
H0A1A1_2b = np.zeros((hf_pt2.nlrg,hf_pt2.nlrg,hf_pt2.nlrg,hf_pt2.nlrg), dtype='complex128')

H1_T1_C1(H0A1A1_1b, None,      H0A1_1b, None,    hf_pt2.T1_1, None,    *fixed_args)
H1_T2_C1(H0A1A1_1b, None,      H0A1_1b, None,    None,    hf_pt2.T2_1, *fixed_args)
H2_T1_C1(H0A1A1_1b, None,      None,    H0A1_2b, hf_pt2.T1_1, None,    *fixed_args)
H2_T2_C1(H0A1A1_1b, None,      None,    H0A1_2b, None,    hf_pt2.T2_1, *fixed_args)
H1_T2_C2(None,      H0A1A1_2b, H0A1_1b, None,    None,    hf_pt2.T2_1, *fixed_args)
H2_T1_C2(None,      H0A1A1_2b, None,    H0A1_2b, hf_pt2.T1_1, None,    *fixed_args)
H2_T2_C2(None,      H0A1A1_2b, None,    H0A1_2b, None,    hf_pt2.T2_1, *fixed_args)
H0A1A1_2b = antisymmetrize_and_hermitize(H0A1A1_2b)
H0A1A1_1b = H0A1A1_1b + H0A1A1_1b.T.conj()
H0A1A1_1b = -H0A1A1_1b
H0A1A1_2b = -H0A1A1_2b


E = (1./6) * H_T_C0(None, None, H0A1A1_1b, H0A1A1_2b, hf_pt2.T1_1, hf_pt2.T2_1, *fixed_args)
print(f'DSRG-MRPT3 energy pt 1: {E:.15f}')

DSRG-MRPT3 energy pt 1: 0.003265049671309-0.000000000000000j


In [1]:
from relforte import *
mol = pyscf.gto.M(
    verbose = 2,
    atom = '''
H 0 0 0
F 0 1.5 0
''',
    basis = 'cc-pvdz', spin=0, charge=0, symmetry=False
)
hf = RelForte(mol, verbose=True, density_fitting=False, decontract=False)
hf.run_rhf(transform=True, debug=False, frozen=(0,0), dump_mo_coeff=False)
hf.run_casci(cas=(6,8), do_fci=False, rdm_level=3, relativistic=False, semi_canonicalize=True)
hf.run_dsrg_mrpt2(s=1.0, relativistic=False, relax='iterate')

              PySCF RHF interface              
Non-relativistic RHF Energy:     -99.8728525 Eh
PySCF RHF time:                    0.4194305 s
-----------------------------------------------
Building integrals...
Integral build time:               0.0246420 s
-----------------------------------------------
RHF time:                          0.4441302 s

          Non-relativistic CASCI(6,8)          
Will now allocate 0.013 MB memory for the CASCI Hamiltonian!
Time taken for 1-RDM build:        0.0073361 s
Time taken for 2-RDM build:        0.0247209 s
Time taken for 3-RDM build:        0.1641724 s
E_frzc:                          -88.7919897 Eh
E_cas:                           -14.2934663 Eh
E_nuc:                             3.1750633 Eh
E_casci:                         -99.9103927 Eh
E0 (from RDM):                   -99.9103927 Eh
Ecorr:                            -0.0375403 Eh
Error to PySCF:                    0.0000000 Eh

Timing summary
... Hamil build:                    0.0187

In [2]:
from relforte import *
mol = pyscf.gto.M(
    verbose = 2,
    atom = '''
H 0 0 0
F 0 1.5 0
''',
    basis = 'cc-pvdz', spin=0, charge=0, symmetry=False
)
hf = RelForte(mol, verbose=True, density_fitting=False, decontract=False)
hf.run_rhf(transform=True, debug=False, frozen=(0,0), dump_mo_coeff=False)
hf.run_casci(cas=(6,8), do_fci=False, rdm_level=3, relativistic=False, semi_canonicalize=True)
hf.run_dsrg_mrpt3(s=1.0, relativistic=False, relax='iterate', pt2_only=True)

              PySCF RHF interface              
Non-relativistic RHF Energy:     -99.8728525 Eh
PySCF RHF time:                    0.3390441 s
-----------------------------------------------
Building integrals...
Integral build time:               0.0223668 s
-----------------------------------------------
RHF time:                          0.3614709 s

          Non-relativistic CASCI(6,8)          
Will now allocate 0.013 MB memory for the CASCI Hamiltonian!
Time taken for 1-RDM build:        0.0064788 s
Time taken for 2-RDM build:        0.0251486 s
Time taken for 3-RDM build:        0.1663568 s
E_frzc:                          -88.7919897 Eh
E_cas:                           -14.2934663 Eh
E_nuc:                             3.1750633 Eh
E_casci:                         -99.9103927 Eh
E0 (from RDM):                   -99.9103927 Eh
Ecorr:                            -0.0375403 Eh
Error to PySCF:                   -0.0000000 Eh

Timing summary
... Hamil build:                    0.0195

In [1]:
from relforte import *
mol = pyscf.gto.M(
    verbose = 2,
    atom = '''
H 0 0 0
F 0 1.5 0
''',
    basis = 'cc-pvdz', spin=0, charge=0, symmetry=False
)
hf = RelForte(mol, verbose=True, density_fitting=False, decontract=False)
hf.run_rhf(transform=True, debug=False, frozen=(0,0), dump_mo_coeff=False)
hf.run_casci(cas=(6,8), do_fci=False, rdm_level=3, relativistic=False, semi_canonicalize=True)
hf.run_dsrg_mrpt3(s=1.0, relativistic=False, relax=3, pt2_only=False)

              PySCF RHF interface              
Non-relativistic RHF Energy:     -99.8728525 Eh
PySCF RHF time:                    0.4111364 s
-----------------------------------------------
Building integrals...
Integral build time:               0.0265958 s
-----------------------------------------------
RHF time:                          0.4377947 s

          Non-relativistic CASCI(6,8)          
Will now allocate 0.013 MB memory for the CASCI Hamiltonian!
Time taken for 1-RDM build:        0.0063989 s
Time taken for 2-RDM build:        0.0249004 s
Time taken for 3-RDM build:        0.1670895 s
E_frzc:                          -88.7919897 Eh
E_cas:                           -14.2934663 Eh
E_nuc:                             3.1750633 Eh
E_casci:                         -99.9103927 Eh
E0 (from RDM):                   -99.9103927 Eh
Ecorr:                            -0.0375403 Eh
Error to PySCF:                    0.0000000 Eh

Timing summary
... Hamil build:                    0.0184

In [2]:
from relforte import *
mol = pyscf.gto.M(
    verbose = 2,
    atom = '''
H 0 0 0
F 0 1.5 0
''',
    basis = 'cc-pvdz', spin=0, charge=0, symmetry=False
)
hf = RelForte(mol, verbose=True, density_fitting=False, decontract=False)
hf.run_rhf(transform=True, debug=False, frozen=(0,0), dump_mo_coeff=False)
hf.run_casci(cas=(6,8), do_fci=False, rdm_level=3, relativistic=False, semi_canonicalize=True)
hf.run_dsrg_mrpt3(s=1.0, relativistic=False, relax='iterate', pt2_only=False)

              PySCF RHF interface              
Non-relativistic RHF Energy:     -99.8728525 Eh
PySCF RHF time:                    0.3768010 s
-----------------------------------------------
Building integrals...
Integral build time:               0.0238726 s
-----------------------------------------------
RHF time:                          0.4007311 s

          Non-relativistic CASCI(6,8)          
Will now allocate 0.013 MB memory for the CASCI Hamiltonian!
Time taken for 1-RDM build:        0.0063190 s
Time taken for 2-RDM build:        0.0264106 s
Time taken for 3-RDM build:        0.1565399 s
E_frzc:                          -88.7919897 Eh
E_cas:                           -14.2934663 Eh
E_nuc:                             3.1750633 Eh
E_casci:                         -99.9103927 Eh
E0 (from RDM):                   -99.9103927 Eh
Ecorr:                            -0.0375403 Eh
Error to PySCF:                    0.0000000 Eh

Timing summary
... Hamil build:                    0.0190

In [17]:
print(f'{hf.e_casci=}')
print(f'{hf.e_dsrg_mrpt2=}')
print(f'{hf.e_dsrg_mrpt3_1=}')
print(f'{hf.e_dsrg_mrpt3_2=}')
print(f'{hf.e_dsrg_mrpt3_3=}')
print(f'{hf.e_dsrg_mrpt3=}')

hf.e_casci=-99.90910395370764
hf.e_dsrg_mrpt2=(-0.19937544936736556+0j)
hf.e_dsrg_mrpt3_1=(0.0025254255329648038-0j)
hf.e_dsrg_mrpt3_2=(-0.005694622162708253+0j)
hf.e_dsrg_mrpt3_3=(-0.00013682821203142792+0j)
hf.e_dsrg_mrpt3=-100.11178542791679
